### Import

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage 
import os
import pandas as pd
import re

#### Input data structure:

In [ ]:
{'question': 'For every 12 ... ', 'answer': 'There are 144/12 = <<144/12=12>>12 ... \n#### 12'}

#### Ex 1:

In [36]:
data = []

with open('math_problems.jsonl', 'r', encoding='utf-8') as f_in:
    for line in f_in:
        data.append(eval(line))

data_df = pd.DataFrame(data)
data_df = data_df.explode(['question', 'answer'])

In [43]:
data_df['target'] = data_df['answer'].apply(lambda x: x.split('####')[-1].strip())

In [44]:
data_df.head()

,question,answer,target
0,"For every 12 cans you recycle, you receive $0....",There are 144/12 = <<144/12=12>>12 sets of 12 ...,12
1,Betty picked 16 strawberries. Matthew picked 2...,Matthew picked 16 + 20 = <<16+20=36>>36 strawb...,40
2,Jack has a stack of books that is 12 inches th...,There are 960 pages because 80 x 12 = <<80*12=...,160
3,James dumps his whole collection of 500 Legos ...,James starts with 500 Legos and uses half of t...,245
4,Ines had $20 in her purse. She bought 3 pounds...,Ines bought 3 pounds of peaches for 3 peaches ...,14


In [57]:
# Initialize the Mistral client with your API key
MODEL = "mistral-small"
TEMPERATURE = 0.0
TOP_P = 1

In [58]:
# Function to open and read the API key file
def open_file(filepath):
    with open(filepath, "r", encoding='utf-8') as infile:
        return infile.read()

# Function to interact with the Mistral API and get a response
def get_mistral_response(user_content):
    
    # Load api key
    api_key = open_file("mistral_api_key.txt")
    client = MistralClient(api_key=api_key)

    # Prepare a List of ChatMessage objects with the user's content
    messages = [ChatMessage(role="user", content=user_content)]
    
    # Get the response from the Mistral API without streaming
    chat_response = client.chat(
        model=MODEL, 
        temperature=TEMPERATURE,
        top_p=TOP_P,
        messages=messages,

        )
    
    try:
        # Extract the content from the response
        response_content = chat_response.choices[0].message.content if chat_response.choices else "" 
    except AttributeError as e:
        print(f"An error occurred while processing the response: {e}")
        response_content = ""
    
    # Return the response content
    return response_content


In [46]:
prompt = {'question': 'For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?', 'answer': 'There are 144/12 = <<144/12=12>>12 sets of 12 cans that the family collected.\nSo, the family would receive $0.50 x 12 = $<<0.50*12=6>>6 for the cans.\nThere are 20/5 = <<20/5=4>>4 sets of 5 kilograms of newspapers that the family collected.\nSo, the family would receive $1.50 x 4 = $<<1.50*4=6>>6 for the newspapers.\nTherefore, the family would receive a total of $6 + $6 = $<<6+6=12>>12.\n#### 12'}
prompt

{'question': 'For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?',
 'answer': 'There are 144/12 = <<144/12=12>>12 sets of 12 cans that the family collected.\nSo, the family would receive $0.50 x 12 = $<<0.50*12=6>>6 for the cans.\nThere are 20/5 = <<20/5=4>>4 sets of 5 kilograms of newspapers that the family collected.\nSo, the family would receive $1.50 x 4 = $<<1.50*4=6>>6 for the newspapers.\nTherefore, the family would receive a total of $6 + $6 = $<<6+6=12>>12.\n#### 12'}

In [65]:
N_RECORD = 10

# Prompt
# user_content = prompt['question']
user_content = data_df.iloc[N_RECORD,0]

# Result
print("Full response:")
response = get_mistral_response(user_content)
print (f"{response}")
print('*'*100)
print("Final result: ")
new_prompt = 'Extract only one number which represent final result from the following text: ' + response
response_sum = re.findall(r'\d+', get_mistral_response(new_prompt))[0]
print (f"{response_sum}")
print('*'*100)
print(f"Expected output: {data_df.iloc[N_RECORD,2]}")

Full response:
John gets 30 cans of soup for the price of 15 cans (buy 1 get one free).

To find out how much John pays, we need to calculate the total cost of the 15 cans at their normal price of $0.60 each.

Total cost = Number of cans * Price per can
Total cost = 15 cans * $0.60/can
Total cost = $9.00

So, John pays $9.00 for 30 cans of soup.
****************************************************************************************************
Final result: 
9
****************************************************************************************************
Expected output: 9


### Ways to reach better performance:
* Ensemble Method: involves combining the outputs of multiple models
* ...